In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
import proj1_helpers as helpers
import models
import costs

In [3]:
DATA_TRAIN_PATH = 'data/train.csv'
y, tXX, ids = helpers.load_csv_data(DATA_TRAIN_PATH)

In [4]:
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
_, tXX_test, ids_test = helpers.load_csv_data(DATA_TEST_PATH)

In [5]:
tX_test = tXX_test.copy()

In [6]:
tX = tXX.copy()

In [7]:
data = np.vstack([tX, tX_test])


## Drope Cells

In [ ]:
l = []
for i in range(tX.shape[1]):
    tmp = tX[:,i]
    c = np.count_nonzero(tmp == -999)
    #print(f"Feature {i} | Occurences: {c} | {c/tX.shape[0]}")
    if c/tX.shape[0] > 0.6:
        l.append(i)

In [70]:
data.shape

(818238, 30)

In [8]:
l = [15, 16, 18, 20]
data = np.delete(data, l, axis=1)
print(data.shape)

(818238, 26)


In [ ]:
plt.hist(tX[:,0])
plt.show()

In [ ]:
y.shape

## Preprocessing

In [8]:
def fix_nonexisting(tX):
    """Replace -999 by median of the row."""
    tX[tX==-999] = np.NaN
    col_median = np.nanmedian(tX, axis=0)
    inds = np.where(np.isnan(tX))
    tX[inds] = np.take(col_median, inds[1])

In [9]:
def normalize(tX):
    a = np.mean(tX, axis=0)
    s = np.std(tX, axis=0)
    tX = (tX-a)/s

In [10]:
def min_max(tX):
    min_ = np.min(tX, axis=0)
    max_ = np.max(tX, axis=0)
    tX = (tX - min_) / (max_ - min_)

In [11]:
def iqr(tX):
    q75, q25 = np.percentile(tX, [75 ,25])
    iqr = q75-q25
    lower = q25 - 1.75 * iqr
    upper = q75 + 1.75 * iqr
    return lower, upper

In [12]:
def remove_outlierts(tX):
    for i in range(tX.shape[1]):
        t = tX[:,i]
        lower, upper = iqr(t)
        t[t < lower] = np.nan
        t[t > upper] = np.nan
        med = np.nanmedian(t)
        t[np.isnan(t)] = med
        tX[:,i] = t
        
def outliers(x, alpha=0):
    """
    Cut the tails: if a value is smaller than alpha_percentile (bigger than 1-alpha_percentile) 
                   of its features replace it with that percentile
    """
    for i in range(x.shape[1]):
        x[:,i][ x[:,i]<np.percentile(x[:,i],alpha) ] = np.percentile(x[:,i],alpha)
        x[:,i][ x[:,i]>np.percentile(x[:,i],100-alpha) ] = np.percentile(x[:,i],100-alpha)
        
    return x

In [13]:
fix_nonexisting(data)
#min_max(data)
#idx = [0,1,2,5,7,9,10,13,16,19,21,23,26]
#a = np.log1p(data[:, idx])
#data = np.hstack((data, a))
#data = outliers(data)
#remove_outlierts(data)
normalize(data)
#l = [15, 16, 18, 20]
#data = np.delete(data, l, axis=1)
tX = data[0:250000]
tX_test = data[250000:]

In [124]:

#remove_outlierts(tX)
#tX = normalize(tX)
#np.mean(tX)

In [22]:
# seed = 1
# degree = 7
# k_fold = 5
# lambdas = [0]
# ks = helpers.build_k_indices(y, k_fold, seed)
print(y)

x_train, x_test, y_train, y_test = helpers.split_data(tX, y, 0.8)
degree = 10
poly_train = models.build_poly(x_train, degree)
poly_test = models.build_poly(x_test, degree)
w = models.ridge_regression(y_train, poly_train, 1125e-10)
y_l = helpers.predict_labels(w, poly_test)
print(y_l)
l = y_test - y_l
np.count_nonzero(l==0)/l.shape[0]

[ 1. -1. -1. ...  1. -1. -1.]
(200000, 301)
[-1. -1. -1. ... -1.  1. -1.]


0.81642

In [13]:
def learning_by_gradient_descent(y, tx, w, gamma):
    """
    Do one step of gradient descent using logistic regression.
    Return the loss and the updated w.
    """
    loss = models.calculate_loss(y, tx, w)
    grad = models.calculate_gradient(y, tx, w)
    w = w - gamma * grad
    return loss, w

In [12]:
def logistic_regression_gradient_descent_demo(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    gamma = -5
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))

    # start the logistic regression
    for iters in range(max_iter):
        #print(iters)
        # get loss and update w.
        loss, w = learning_by_gradient_descent(y, tx, w, gamma)
        # log info
        if iters % 10 == 0:
            print("Current iteration={i}, loss={l}".format(i=iters, l=loss))
        # converge criterion



In [14]:
y[y==-1] = 0

In [15]:
logistic_regression_gradient_descent_demo(y, tX)

In [1]:
tX[:100,:].shape

NameError: name 'tX' is not defined

## Generate predictions and save ouput in csv format for submission:

In [1]:
DATA_TEST_PATH = 'data/test.csv' # TODO: download train data and supply path here 
_, tXX_test, ids_test = helpers.load_csv_data(DATA_TEST_PATH)
tX_test = tXX_test.copy()

NameError: name 'helpers' is not defined

In [145]:
m, s = normalize(np.vstack([tX, tX_test]))
tX = (tX-m)/s
tX_test = (tX_test-m)/s

In [65]:
OUTPUT_PATH = 'data/sample-submission.csv' # TODO: fill in desired name of output file for submission
#fix_nonexisting(tX_test)
#tX_test = normalize(tX_test)
#remove_outlierts(tX_test)

In [85]:
poly = models.build_poly(tX_test, 10)
y_pred = helpers.predict_labels(w, poly)
helpers.create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [86]:
from tester import tester

tester(OUTPUT_PATH)

0.8171417610226701